Retrieval Augmented Generation (RAG) solves this by retrieving relevant data and injecting it into the prompt.

In this notebook, we will learn:

Embeddings: Representing text as vectors.
Vector Stores: Storing and searching vectors (FAISS).
Naïve RAG: The standard Retrieval -> Augment -> Generate pipeline.
Indexing Challenges: Deep dive into how vector databases search efficiently (Flat, IVF, HNSW, PQ).

STEP 1:
Embeddings:
An embedding is a translation from Words to Lists of Numbers (Vectors), such that similar words represent close numbers.

The Process (Flowchart)
graph LR
    A["Input Text ('Apple')"] -->|Tokenization| B["Tokens (101, 255)"]
    B -->|Embedding Model| C["Vector List ([0.1, -0.5, 0.9...])"]
    C -->|Store| D["Vector Database"]

In [2]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-huggingface sentence-transformers langchain-community

from dotenv import load_dotenv
load_dotenv()

import os
from langchain_huggingface import HuggingFaceEmbeddings

# Using a FREE, open-source model from Hugging Face
# 'all-MiniLM-L6-v2' is small, fast, and very good for English.
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

2. Viewing a Vector
Let's see what the word "Apple" looks like to the machine.

Conceptual Note: Dimensions
The vector below has 384 dimensions (for MiniLM).

Imagine a graph with X and Y axes (2 Dimensions). You can plot a point (x, y).
Now imagine adding Z (3 Dimensions).
Now imagine 384 axes.
Each axis represents a feature (e.g., "Is it a fruit?", "Is it red?", "Is it tech-related?"). The numbers aren't random; they encode meaning.

In [3]:
vector = embeddings.embed_query("Apple")

print(f"Dimensionality: {len(vector)}")
print(f"First 5 numbers: {vector[:5]}")

Dimensionality: 384
First 5 numbers: [-0.006138487718999386, 0.03101177327334881, 0.06479360908269882, 0.01094149798154831, 0.005267191678285599]


step 3:  Cosine Similarity
How do we know if two vectors are close? We measure the Angle between them.

Cosine Similarity Formula


1.0: Arrows point in the Exact Same Direction (Identical).
0.0: Arrows are Perpendicular (Unrelated).
-1.0: Arrows point in Opposite Directions (Opposite).
Experiment: Let's compare "Cat", "Dog", and "Car".

In [4]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

vec_cat = embeddings.embed_query("Cat")
vec_dog = embeddings.embed_query("Dog")
vec_car = embeddings.embed_query("Car")

print(f"Cat vs Dog: {cosine_similarity(vec_cat, vec_dog):.4f}")
print(f"Cat vs Car: {cosine_similarity(vec_cat, vec_car):.4f}")

Cat vs Dog: 0.6606
Cat vs Car: 0.4633


Analysis
You should see that Cat & Dog score higher (e.g., ~0.8) than Cat & Car (e.g., ~0.3).
This Mathematical Distance is the foundation of all Search engines and RAG systems.
This is arguably the most important concept in modern AI.

Unit 2 - Part 4b: Naive RAG Pipeline
1. Introduction: The Open-Book Test
RAG (Retrieval-Augmented Generation) is just an Open-Book Test architecture.

Retrieval: Find the right page in the textbook.
Generation: Write the answer using that page.
The Pipeline (Flowchart)
graph TD
    User[User Question] --> Retriever[Retriever System]
    Retriever -->|Search Database| Docs[Relevant Documents]
    Docs --> Combiner[Prompt Template]
    User --> Combiner
    Combiner -->|Full Prompt w/ Context| LLM[Gemini Model]
    LLM --> Answer[Final Answer]

In [6]:
# Setup
%pip install python-dotenv --upgrade --quiet faiss-cpu langchain-huggingface sentence-transformers langchain-community langchain-google-genai
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Using the same free model as Part 4a
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.0 MB/s eta 0:00:00


step 2
The "Knowledge Base" (Grounding)
LLMs hallucinate because they rely on "parametric memory" (what they learned during training). RAG introduces "non-parametric memory" (external facts).

Let's define some facts the LLM definitely does not know.

In [7]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Ram's favorite food is Pizza with extra cheese."),
    Document(page_content="The secret password to the lab is 'Blueberry'."),
    Document(page_content="LangChain is a framework for developing applications powered by language models."),
]

3. Indexing ( Storing the knowledge)
We use FAISS (Facebook AI Similarity Search) to store the embeddings. Think of FAISS as a super-fast librarian that organizes books by content, not title.

In [8]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

4. The RAG Chain
We use LCEL to stitch it together.

Step 1: The retriever takes the question, converts it to numbers, and finds the closest document. Step 2: RunnablePassthrough holds the question. Step 3: The prompt combines them.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
Answer based ONLY on the context below:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("What is the secret password?")
print(result)

The secret password to the lab is 'Blueberry'.


Unit 2 - Part 4c: Deep Dive into Indexing Algorithms
1. Introduction: The Scale Problem
Comparing 1 vector against 10 vectors is fast. Comparing 1 vector against 100 Million vectors is slow.

FAISS (Facebook AI Similarity Search) was built to solve this.

The Trade-off Triangle
You can pick 2:

Speed (Query time)
Accuracy (Recall)
Memory (RAM usage)
We will explore algorithms that optimize different corners of this triangle.

FAISS
To find similar items:
Compare query with ALL vectors → VERY SLOW

With FAISS:
Use smart indexing → VERY FAST search

FAISS = Toolbox
IVF, Flat, HNSW = Tools inside the toolbox
FAISS
| Data Type       | Converted to vector using |
| --------------- | ------------------------- |
| Text            | BERT                      |
| Image           | CLIP                      |
| Audio           | Wav2Vec                   |
| Medical records | ML models                 |
Raw Data → ML Model → Embeddings → FAISS Index → Fast Search

| Inside FAISS | Type              |
| ------------ | ----------------- |
| IndexFlatL2  | Brute force       |
| IVF          | Clustering based  |
| HNSW         | Graph based       |
| PQ           | Compression based |




In [10]:
import faiss
import numpy as np

# Mock Data: 10,000 vectors of size 128
d = 128
nb = 10000
xb = np.random.random((nb, d)).astype('float32')

2. Flat Index (Brute Force)
Concept: Check every single item.

Algo: IndexFlatL2
Pros: 100% Accuracy (Gold Standard).
Cons: Slow (O(N)). Unusable at 1M+ vectors.

In [11]:
index = faiss.IndexFlatL2(d)
index.add(xb)
print(f"Flat Index contains {index.ntotal} vectors")
xq = np.random.random((1, d)).astype('float32')
k = 5
D, I = index.search(xq, k)

print("Nearest vector indices:", I)
print("Distances:", D)


Flat Index contains 10000 vectors
Nearest vector indices: [[2566  483 5435 6119  859]]
Distances: [[13.689927 14.276472 14.403429 14.410736 14.534609]]


In [12]:
vector_0 = index.reconstruct(0)
print(vector_0[:10])


[0.2580179  0.98567814 0.24375844 0.8135036  0.14280279 0.740962
 0.70723194 0.3000329  0.33059973 0.9030029 ]


IVF (Inverted File Index)
Concept: Clustering / Partitioning.

Imagine looking for a book. Instead of checking every shelf, you go to the "Sci-Fi" section. Then you only search books in that section.

How it works (Flowchart)
graph TD
    Data[All 1M Vectors] -->|Train| Clusters[1000 Cluster Centers (Centroids)]
    Query[User Query] -->|Step 1| FindClosest[Find Closest Centroid]
    FindClosest -->|Step 2| Search[Search ONLY vectors in that Cluster]
Analogy: Voronoi Cells (Zip Codes). We only search the local zip code.

In [13]:
nlist = 100 # How many 'zip codes' (clusters) we want
quantizer = faiss.IndexFlatL2(d) # The calculator for distance
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)

# We MUST train it first so it learns where the clusters are
index_ivf.train(xb)
index_ivf.add(xb)

In [14]:
print("Is index trained?", index_ivf.is_trained)
print("Total vectors in index:", index_ivf.ntotal)
print("Number of clusters (nlist):", index_ivf.nlist)


Is index trained? True
Total vectors in index: 10000
Number of clusters (nlist): 100


In [15]:
index_ivf.nprobe = 5   # search in 5 clusters

xq = np.random.random((1, d)).astype('float32')
D, I = index_ivf.search(xq, 5)

print("Nearest indices:", I)
print("Distances:", D)


Nearest indices: [[3214 3550  891 5483  846]]
Distances: [[13.808525 13.945784 14.322535 14.37821  14.569209]]


In [16]:
index_ivf.nprobe = 5   # search in 5 clusters

xq = np.random.random((1, d)).astype('float32')
D, I = index_ivf.search(xq, 5)

print("Nearest indices:", I)
print("Distances:", D)


Nearest indices: [[4227 3368 9378 5290 8348]]
Distances: [[14.8670845 15.842625  15.96616   16.341606  16.519085 ]]


4. HNSW (Hierarchical Navigable Small World)
Concept: Six Degrees of Separation.

Most data is connected. HNSW builds a Graph.

Layer 0: Every point connects to neighbors.
Layer 1: "Express Highways" connecting distant points.
Analogy: Catching a flight. You don't fly Local -> Local -> Local. You fly Local -> HUB (Chicago) -> HUB (London) -> Local.

Pros: Extremely fast retrieval.
Cons: Heavier on RAM (needs to store the edges of the graph).

In [17]:
M = 16 # Number of connections per node (The 'Hub' factor)
index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.add(xb)

In [18]:
xq = np.random.random((1, d)).astype('float32')

D, I = index_hnsw.search(xq, 5)

print("Nearest indices:", I)
print("Distances:", D)


Nearest indices: [[3722 4980 9386 7035 8119]]
Distances: [[14.36725  15.214366 15.394323 15.525642 15.611689]]


5. PQ (Product Quantization)
Concept: Compression (Lossy).

Do we need 32-bit float precision (0.123456789)? No. 0.12 is fine. PQ breaks the vector into chunks and approximates them.

Analogy: 4K Video vs 480p Video.

480p is blurry, but it's 10x smaller and faster to stream.
Use PQ when you are RAM constrained (e.g., storing 1 Billion vectors)

In [19]:
m = 8 # Split vector into 8 sub-vectors
index_pq = faiss.IndexPQ(d, m, 8)
index_pq.train(xb)
index_pq.add(xb)
print("PQ Compression complete. RAM usage minimized.")

PQ Compression complete. RAM usage minimized.


| Index | Speed     | Accuracy  | Memory   |
| ----- | --------- | --------- | -------- |
| Flat  | Slow      | 100%      | High     |
| IVF   | Fast      | High      | Medium   |
| HNSW  | Very Fast | Very High | High     |
| PQ    | Very Fast | Medium    | Very Low |

| Method | Think as        |
| ------ | --------------- |
| Flat   | Check All       |
| IVF    | Go to Section   |
| HNSW   | Travel via Hubs |
| PQ     | Compress Data   |

Flat → Exact but heavy
IVF → Clustered search
HNSW → Graph navigation
PQ → Compressed storage

